<a href="https://colab.research.google.com/github/blankon123/random-analytics/blob/main/Pandas_n_Scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import sklearn as sk
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [3]:
tpk_harian = pd.read_csv('/content/drive/MyDrive/bigdata-bps-hackathon-2021/train-online_booking_2020.csv',parse_dates=['tanggal'])
tpk_bps = pd.read_csv('/content/drive/MyDrive/bigdata-bps-hackathon-2021/train-TPK_Hotel_berbintang_2020.csv')

In [4]:
tpk_harian.head()

,province,type,tanggal,klasifikasi,all_available_room,room_total
0,Bali,Hotel,2020-01-01,Rated,2155,37756
1,Bali,Hotel,2020-01-01,Not Rated,163,392
2,Bali,Hotel,2020-01-02,Rated,3878,38814
3,Bali,Hotel,2020-01-02,Not Rated,211,415
4,Bali,Hotel,2020-01-03,Rated,5412,39084


In [5]:
tpk_bps.head()

,Id,Provinsi,Tahun,Bulan,Aggregate_var,TPK
0,1,Bali,2020,Januari,NaN,59.29
1,2,Bali,2020,Februari,NaN,45.98
2,3,Bali,2020,Maret,NaN,25.41
3,4,Bali,2020,April,NaN,3.22
4,5,Bali,2020,Mei,NaN,2.07


## Hitung TPK Harian

In [6]:
tpk_harian['used_room'] = tpk_harian.room_total-tpk_harian.all_available_room

In [7]:
tpk_harian['tpk_online'] = tpk_harian.used_room/tpk_harian.room_total*100

In [8]:
tpk_harian.head()

,province,type,tanggal,klasifikasi,all_available_room,room_total,used_room,tpk_online
0,Bali,Hotel,2020-01-01,Rated,2155,37756,35601,94.292298
1,Bali,Hotel,2020-01-01,Not Rated,163,392,229,58.418367
2,Bali,Hotel,2020-01-02,Rated,3878,38814,34936,90.008760
3,Bali,Hotel,2020-01-02,Not Rated,211,415,204,49.156627
4,Bali,Hotel,2020-01-03,Rated,5412,39084,33672,86.152901


## Agregat Bulanan

In [9]:
tpk_harian = tpk_harian.set_index('tanggal')

In [10]:
tpk_harian_agg = tpk_harian.groupby(pd.Grouper(freq='M')).mean()

Sementara pake Mean, jika dirasa ada metode agregasi lain yang lebih mewakili, bisa dicoba disini

In [11]:
tpk_harian_agg = tpk_harian_agg.reset_index()
tpk_harian_agg

,tanggal,all_available_room,room_total,used_room,tpk_online
0,2020-01-31,5443.816667,21726.200000,16282.383333,57.250772
1,2020-02-29,7688.804348,23770.695652,16081.891304,54.108035
2,2020-03-31,15901.550000,27909.350000,12007.800000,32.692866
3,2020-04-30,16367.783333,21969.950000,5602.166667,21.881438
4,2020-05-31,15812.822581,24101.661290,8288.838710,28.758468
5,2020-06-30,17156.333333,25541.133333,8384.800000,23.549558
6,2020-07-31,16859.032258,24627.903226,7768.870968,23.315312
7,2020-08-31,15173.451613,24325.645161,9152.193548,28.478181
8,2020-09-30,16161.720000,26621.660000,10459.940000,26.950464
9,2020-10-31,16287.750000,26245.216667,9957.466667,27.025837


## Join Dataframe

In [13]:
tpk_harian_agg['Id'] = np.arange(start=1, stop=13, step=1)
tpk_harian_agg.head()

,tanggal,all_available_room,room_total,used_room,tpk_online,Id
0,2020-01-31,5443.816667,21726.200000,16282.383333,57.250772,1
1,2020-02-29,7688.804348,23770.695652,16081.891304,54.108035,2
2,2020-03-31,15901.550000,27909.350000,12007.800000,32.692866,3
3,2020-04-30,16367.783333,21969.950000,5602.166667,21.881438,4
4,2020-05-31,15812.822581,24101.661290,8288.838710,28.758468,5


In [14]:
tpk_join = pd.merge(tpk_bps,tpk_harian_agg,on='Id',how='left')
tpk_join.head()

,Id,Provinsi,Tahun,Bulan,Aggregate_var,TPK,tanggal,all_available_room,room_total,used_room,tpk_online
0,1,Bali,2020,Januari,NaN,59.29,2020-01-31,5443.816667,21726.200000,16282.383333,57.250772
1,2,Bali,2020,Februari,NaN,45.98,2020-02-29,7688.804348,23770.695652,16081.891304,54.108035
2,3,Bali,2020,Maret,NaN,25.41,2020-03-31,15901.550000,27909.350000,12007.800000,32.692866
3,4,Bali,2020,April,NaN,3.22,2020-04-30,16367.783333,21969.950000,5602.166667,21.881438
4,5,Bali,2020,Mei,NaN,2.07,2020-05-31,15812.822581,24101.661290,8288.838710,28.758468


In [15]:
tpk_join = tpk_join.drop(['tanggal','Aggregate_var','Provinsi'],axis=1)

Drop beberapa column yang dinilai kurang informatif untuk modelling

## Transformasi

In [25]:
from sklearn.preprocessing import StandardScaler

std_scaler_tpk = StandardScaler()
tpk_join['tpk_bps_standard'] = std_scaler_tpk.fit_transform(tpk_join['TPK'].values.reshape(-1,1))

std_scaler_tpk_online = StandardScaler()
tpk_join['tpk_online_standard'] = std_scaler_tpk_online.fit_transform(tpk_join['tpk_online'].values.reshape(-1,1))

tpk_join.head()

,Id,Tahun,Bulan,TPK,all_available_room,room_total,used_room,tpk_online,tpk_bps_standard,tpk_online_standard
0,1,2020,Januari,59.29,5443.816667,21726.200000,16282.383333,57.250772,2.405994,2.271070
1,2,2020,Februari,45.98,7688.804348,23770.695652,16081.891304,54.108035,1.672709,1.986072
2,3,2020,Maret,25.41,15901.550000,27909.350000,12007.800000,32.692866,0.539450,0.044045
3,4,2020,April,3.22,16367.783333,21969.950000,5602.166667,21.881438,-0.683059,-0.936385
4,5,2020,Mei,2.07,15812.822581,24101.661290,8288.838710,28.758468,-0.746415,-0.312744


In [26]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler_tpk = MinMaxScaler()
tpk_join['tpk_bps_minmax'] = minmax_scaler_tpk.fit_transform(tpk_join['TPK'].values.reshape(-1,1))

minmax_scaler_tpk_online = MinMaxScaler()
tpk_join['tpk_online_minmax'] = minmax_scaler_tpk_online.fit_transform(tpk_join['tpk_online'].values.reshape(-1,1))

tpk_join.head()

,Id,Tahun,Bulan,TPK,all_available_room,room_total,used_room,tpk_online,tpk_bps_standard,tpk_online_standard,tpk_bps_minmax,tpk_online_minmax
0,1,2020,Januari,59.29,5443.816667,21726.200000,16282.383333,57.250772,2.405994,2.271070,1.000000,1.000000
1,2,2020,Februari,45.98,7688.804348,23770.695652,16081.891304,54.108035,1.672709,1.986072,0.767389,0.911145
2,3,2020,Maret,25.41,15901.550000,27909.350000,12007.800000,32.692866,0.539450,0.044045,0.407899,0.305672
3,4,2020,April,3.22,16367.783333,21969.950000,5602.166667,21.881438,-0.683059,-0.936385,0.020098,0.000000
4,5,2020,Mei,2.07,15812.822581,24101.661290,8288.838710,28.758468,-0.746415,-0.312744,0.000000,0.194435


## Export Dataframe

In [27]:
tpk_join.head()

,Id,Tahun,Bulan,TPK,all_available_room,room_total,used_room,tpk_online,tpk_bps_standard,tpk_online_standard,tpk_bps_minmax,tpk_online_minmax
0,1,2020,Januari,59.29,5443.816667,21726.200000,16282.383333,57.250772,2.405994,2.271070,1.000000,1.000000
1,2,2020,Februari,45.98,7688.804348,23770.695652,16081.891304,54.108035,1.672709,1.986072,0.767389,0.911145
2,3,2020,Maret,25.41,15901.550000,27909.350000,12007.800000,32.692866,0.539450,0.044045,0.407899,0.305672
3,4,2020,April,3.22,16367.783333,21969.950000,5602.166667,21.881438,-0.683059,-0.936385,0.020098,0.000000
4,5,2020,Mei,2.07,15812.822581,24101.661290,8288.838710,28.758468,-0.746415,-0.312744,0.000000,0.194435


In [28]:
tpk_join.to_csv('TPK_JOIN_PREPROC_V2.csv')

## EDA

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=tpk_join['Id'], y=tpk_join['TPK'],
                    mode='lines+markers',
                    name='TPK BPS'))
fig.add_trace(go.Scatter(x=tpk_join['Id'], y=tpk_join['tpk_online'],
                    mode='lines+markers',
                    name='TPK Online'))

fig.show()